In [ ]:
import pprint
import google.generativeai as palm

PALM_API=""
API=""

palm.configure(api_key=PALM_API)


import google.generativeai as genai
genai.configure(api_key=API)

from langchain.llms import GooglePalm
from langchain.embeddings import HuggingFaceInstructEmbeddings

llm_palm=GooglePalm(google_api_key=PALM_API,temperature=0.003)

def getpalmres(prompt):
    #print("ther")
    result=llm_palm(prompt)
    return result


def getmodel():
    return genai.GenerativeModel("gemini-pro")


def getvision():
    return genai.GenerativeModel("gemini-pro-vision")


def queryimage(mdl,prompt,img):
    response=mdl.generate_content([prompt,img])
    response.resolve()
    return (response.text)
def querytext(mdl,prompt):
    #print(prompt)
    response=mdl.generate_content(prompt)
    return (response.text)  

model=getmodel()
vision=getvision()

In [ ]:
import regex as re
import time
#import pywhatkit
from datetime import datetime

from selenium import webdriver

import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select, WebDriverWait
import os

import copy
from bs4 import BeautifulSoup
import requests 
from PIL import Image 

In [ ]:
driver = webdriver.Chrome()
driver.get("https://instagram.com")

print("Scan QR Code, And then Enter")
input()
print("Logged In")

In [ ]:

name="Sudheesh Srivastava"


In [ ]:
def getmessage(driver,name):
    #assumes driver is the DM page of contact open
    messages=driver.find_element(By.XPATH,f'//*[@aria-label="Messages in conversation with {name}"]')
    soup=BeautifulSoup(messages.find_elements(By.XPATH,'//*[@role="row"]')[-1].get_attribute('innerHTML'))
    txt=soup.get_text()
    auth=''
    #print(txt)
    if txt.find(name)>=0:
        txt=txt.replace(name,"").replace("Enter","")
        #if txt.find("Batty:")>=0:
            #res=getpalmres(f"{txt[txt.find('Batty:')+6:]}")
            #sendmessage(driver,res)
        auth=name
    else:
        txt=txt.replace("You","").replace("sent","").replace("Enter","")
        auth='You'
    return txt,auth

def sendmessage(driver,text):
    input=driver.find_element(By.XPATH,'//*[@aria-label="Message"]')
    input.send_keys(text+Keys.RETURN)

def getimage(driver,name):
    messages=driver.find_element(By.XPATH,f'//*[@aria-label="Messages in conversation with {name}"]') 
    soup=BeautifulSoup(messages.find_elements(By.XPATH,'//*[@role="row"]')[-1].get_attribute('innerHTML'))
    url=soup.find_all("img",{"alt":"Open photo"})[0]['src']
    
    data = requests.get(url).content 
    open('img.jpg','wb').write(data)
    img=Image.open("img.jpg")

    return img

In [ ]:
getmessage(driver,name)

In [ ]:
while True:
    name="Sudheesh Srivastava"
    try:
        txt,auth=getmessage(driver,name)
        if auth.find(name)>=0:
            if txt.find("Batty:")>=0:
                query=txt[txt.find('Batty:')+6:]
                if query.find("<EXIT>")>=0:
                    break
                else:
                    print(f"Getting response from PaLM2")
                    res=getpalmres(query)
                    sendmessage(driver,res)
            elif txt.find("Gobhi:")>=0:
                query=txt[txt.find('Gobhi:')+6:]
                sendmessage(driver,"WAITING 5 secs for IMAGE")
                
                try:
                    time.sleep(5)
                    print('a')
                    img=getimage(driver,name)
                    print('b')
                    res=queryimage(vision,query,img)
                    print(res)
                    sendmessage(driver,res)
                except:
                    print("ERROR IN GETTING IMAGE QUERY")
                    sendmessage(driver,"ERROR IN GETTING IMAGE QUERY")
            else:
                pass
    except:
        print("ERROR")
        pass
    time.sleep(3)